In [14]:
# YOUR IMPORTS HERE
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation

import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer
from torch.utils.data import DataLoader

In [15]:
train_data = np.load('/kaggle/input/mlprojectdata/train_images.npy')
train_labels = np.load('/kaggle/input/mlprojectdata/train_labels.npy')
val_data = np.load('/kaggle/input/mlprojectdata/val_images.npy')
val_labels = np.load('/kaggle/input/mlprojectdata/val_labels.npy')

In [31]:
# custom dataset object for dataloaders

class My_Dataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], int(self.y_data[index][0])
        
    def __len__ (self):
        return len(self.X_data)

train_data_for_loader = My_Dataset(torch.FloatTensor(train_data), train_labels)
val_data_for_loader = My_Dataset(torch.FloatTensor(val_data), val_labels)

train_loader = DataLoader(train_data_for_loader, batch_size = 100, shuffle = True)
val_loader = DataLoader(val_data_for_loader, batch_size = len(val_data_for_loader), shuffle = False)

NameError: name 'Dataset' is not defined

In [27]:
# define and instantiate model

class Model(nn.Module):
    # YOUR CODE HERE
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Model()

In [28]:
# define and instantiate loss function & optimizer

# YOUR CODE HERE
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [29]:
# train model

# YOUR CODE HERE
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: Given groups=1, weight of size [6, 3, 5, 5], expected input[1, 100, 28, 28] to have 3 channels, but got 100 channels instead

In [ ]:
# check accuracy on entire test set

# count correct predictions
with torch.no_grad():
    correct = 0
    for X_test, y_test in test_loader:
        y_pred = model(X_test.view(len(X_test), -1))
        predicted = torch.max(y_pred, 1)[1]
        correct += (predicted == y_test).sum()

# calculate accuracy, print
print(f' Accuracy: {correct.item() / len(test_data_for_loader)}')

In [ ]:
# FUNCTION TO SAVE PREDICTIONS TO CSV
# call with trained model and DataLoader for test data
# will save predictions to .csv in current directory
def save_predictions(M, T):
    with open('submission.csv', 'w') as out_file:
        s = 'Id,Category\n'
        with torch.no_grad():
            for X_test, y_test in T:
                y_pred = model(X_test.view(len(X_test), -1))
                predicted = torch.max(y_pred, 1)[1]
                for i in range(len(predicted)):
                    s += f'{i},{str(int(predicted[i]))}\n'
        s = s[:-1]
        out_file.write(s)
        
save_predictions(model, test_loader)